In [1]:
import glob
import os
import shutil

from astropy.io import fits
from astropy import stats
from astropy.time import Time
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Determine where the visualization will be rendered
output_notebook() 

Loading BokehJS ...

In [4]:
postflash_data = pd.read_pickle('Feb_2022_flc_all_stats_postflash.pkl')

In [5]:
postflash_data[0:10]

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,median,sig_median,std,sig_std,norm_by_dur,datetime
rootname,,,,,,,,,,,,,,,,,,,
ic6de2u9q,ic6de2u9q_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-03-09,13:59:17,B,12,4.7,LOW,True,0.5,7.312437,7.048593,6.917119,6.917119,17.943027,2.911503,1.499701,2013-03-09 13:59:17
icha24imq,icha24imq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-06-29,04:16:36,A,3,1.2,LOW,True,0.5,2.170188,1.976714,1.933412,1.933412,15.109935,2.360522,1.647262,2014-06-29 04:16:36
idp530aiq,idp530aiq_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2018-06-22,04:55:12,B,15,0.2,MED,True,0.5,8.972477,8.680973,8.854218,8.854218,22.985115,3.162331,43.404864,2018-06-22 04:55:12
ieg845feq,ieg845feq_flc.fits,16398,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-12-14,20:55:49,B,7310,100.0,MED,False,0.5,4685.206055,4683.406877,4759.056641,4759.056641,436.754791,419.022524,46.834069,2020-12-14 20:55:49
icrz19f7q,icrz19f7q_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-05-25,22:31:24,B,12,4.7,LOW,True,0.5,7.337009,7.018181,6.969007,6.969007,27.715347,2.952669,1.493230,2015-05-25 22:31:24
idus35y6q,idus35y6q_flc.fits,15573,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2019-08-15,10:09:26,A,7310,100.0,MED,False,0.5,4876.872559,4875.340172,4947.657715,4947.657715,436.621338,430.077256,48.753402,2019-08-15 10:09:26
ie1b44diq,ie1b44diq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2019-12-09,09:02:57,B,22,0.3,MED,False,100.0,16.624331,14.670476,14.623810,14.623810,78.962532,3.976113,48.901587,2019-12-09 09:02:57
idp531a7q,idp531a7q_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2018-07-20,04:24:43,B,12,4.7,LOW,True,0.5,7.372123,7.059274,6.966012,6.966012,18.031178,2.999503,1.501973,2018-07-20 04:24:43
idp543c4q,idp543c4q_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2017-12-11,05:29:33,B,12,4.7,LOW,False,0.5,8.491176,7.938883,7.974644,7.974644,52.009426,3.177174,1.689124,2017-12-11 05:29:33


In [8]:
subarray_pf = postflash_data.loc[(postflash_data['subarray'] == True)] 
fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)]
low_sub_pf_A = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A')] 
low_sub_pf_B = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B')] 

In [9]:
fullframe_pf_A = postflash_data.loc[(postflash_data['flash_cur'] == 'MED') & (postflash_data['subarray'] == False) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 7310)]

In [10]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(fullframe_pf_A['obsdate'] + ' ' + fullframe_pf_A['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(fullframe_pf_A['norm_by_dur'])
y = fullframe_pf_A['norm_by_dur']/fullframe_pf_A['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation and Duration',x_axis_type='datetime',plot_width=800, plot_height=300)
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Fullframe, Shutter A")
p.line(x_datetime,y_pred,color='red')#,legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -5.567769267852128e-06


In [11]:
fullframe_pf_B = postflash_data.loc[(postflash_data['flash_cur'] == 'MED') & (postflash_data['subarray'] == False) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 7310)]

In [12]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(fullframe_pf_B['obsdate'] + ' ' + fullframe_pf_B['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(fullframe_pf_B['norm_by_dur'])
y = fullframe_pf_B['norm_by_dur']/fullframe_pf_B['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation and Duration',x_axis_type='datetime',plot_width=800, plot_height=300)
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Fullframe, Shutter B")
p.line(x_datetime,y_pred,color='red')#,legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -5.909618383346944e-06


In [9]:
low_sub_pf_A_lvl_3 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 3)]
low_sub_pf_A_lvl_3

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,median,sig_median,std,sig_std,norm_by_dur,datetime
rootname,,,,,,,,,,,,,,,,,,,
icha24imq,icha24imq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-06-29,04:16:36,A,3,1.2,LOW,True,0.5,2.170188,1.976714,1.933412,1.933412,15.109935,2.360522,1.647262,2014-06-29 04:16:36
icrz24uhq,icrz24uhq_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-06-23,19:44:30,A,3,1.2,LOW,True,0.5,2.378808,1.928233,1.852256,1.852256,30.059950,2.389974,1.606861,2015-06-23 19:44:30
idcd42hkq,idcd42hkq_flc.fits,14535,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2017-10-10,03:49:32,A,3,1.2,LOW,True,0.5,2.379009,2.021952,1.968841,1.968841,38.600693,2.469626,1.684960,2017-10-10 03:49:32
ie1b39j2q,ie1b39j2q_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-09-08,09:29:23,A,3,1.3,LOW,True,0.5,2.352620,2.125566,2.072635,2.072635,17.584522,2.562709,1.635051,2020-09-08 09:29:23
idcd07o5q,idcd07o5q_flc.fits,14535,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2016-12-07,06:58:42,A,3,1.2,LOW,True,0.5,2.322648,1.953961,1.943979,1.943979,35.924011,2.434325,1.628301,2016-12-07 06:58:42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
icrz15r3q,icrz15r3q_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-04-01,19:41:03,A,3,1.2,LOW,True,0.5,2.152912,1.924168,1.858017,1.858017,14.567774,2.385475,1.603473,2015-04-01 19:41:03
idus12phq,idus12phq_flc.fits,15573,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2019-01-02,13:31:24,A,3,1.2,LOW,True,0.5,2.163204,1.951422,1.896657,1.896657,15.860590,2.521220,1.626185,2019-01-02 13:31:24
icrz06isq,icrz06isq_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-01-06,09:47:56,A,3,1.2,LOW,True,0.5,2.312021,1.991992,2.002017,2.002017,28.837175,2.378134,1.659993,2015-01-06 09:47:56


In [10]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_3['obsdate'] + ' ' + low_sub_pf_A_lvl_3['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_3['sig_mean'])
y = low_sub_pf_A_lvl_3['sig_mean']/low_sub_pf_A_lvl_3['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print(popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean of First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 3 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

1.7225596203505373e-05


In [11]:
low_sub_pf_B_lvl_3 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 3)]

In [12]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_3['obsdate'] + ' ' + low_sub_pf_B_lvl_3['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_3['sig_mean'])
y = low_sub_pf_B_lvl_3['sig_mean']/low_sub_pf_B_lvl_3['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 3 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  1.6698238698545852e-05


In [13]:
low_sub_pf_A_lvl_5 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 5)]

# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_5['obsdate'] + ' ' + low_sub_pf_A_lvl_5['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_5['sig_mean'])
y = low_sub_pf_A_lvl_5['sig_mean']/low_sub_pf_A_lvl_5['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 5 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  7.956740081994086e-06


In [14]:
low_sub_pf_B_lvl_5 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 5)]
low_sub_pf_A_lvl_8 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 8)]
low_sub_pf_B_lvl_8 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 8)]
low_sub_pf_A_lvl_10 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 10)]
low_sub_pf_B_lvl_10 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 10)]
low_sub_pf_A_lvl_12 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 12)]
low_sub_pf_B_lvl_12 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 12)]
low_sub_pf_A_lvl_20 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 20)]
low_sub_pf_B_lvl_20 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 20)]


In [7]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_5['obsdate'] + ' ' + low_sub_pf_B_lvl_5['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_5['sig_mean'])
y = low_sub_pf_B_lvl_5['sig_mean']/low_sub_pf_B_lvl_5['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 5 Shutter B")
p.line(x_datetime,y_pred,color='red', legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  1.0069997793520035e-05


In [8]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_8['obsdate'] + ' ' + low_sub_pf_A_lvl_8['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_8['sig_mean'])
y = low_sub_pf_A_lvl_8['sig_mean']/low_sub_pf_A_lvl_8['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 8 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  1.1063088721121987e-05


In [9]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_8['obsdate'] + ' ' + low_sub_pf_B_lvl_8['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_8['sig_mean'])
y = low_sub_pf_B_lvl_8['sig_mean']/low_sub_pf_B_lvl_8['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 8 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  1.2878047858413839e-05


In [10]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_10['obsdate'] + ' ' + low_sub_pf_A_lvl_10['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_10['sig_mean'])
y = low_sub_pf_A_lvl_10['sig_mean']/low_sub_pf_A_lvl_10['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 10 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  1.5660967744858843e-05


In [11]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_10['obsdate'] + ' ' + low_sub_pf_B_lvl_10['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_10['sig_mean'])
y = low_sub_pf_B_lvl_10['sig_mean']/low_sub_pf_B_lvl_10['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 10 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  1.3986637953236158e-05


In [12]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_12['obsdate'] + ' ' + low_sub_pf_A_lvl_12['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_12['sig_mean'])
y = low_sub_pf_A_lvl_12['sig_mean']/low_sub_pf_A_lvl_12['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 12 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  8.348776030904217e-06


In [13]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_12['obsdate'] + ' ' + low_sub_pf_B_lvl_12['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_12['sig_mean'])
y = low_sub_pf_B_lvl_12['sig_mean']/low_sub_pf_B_lvl_12['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.1)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 12 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  8.380404998198067e-06


In [14]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_20['obsdate'] + ' ' + low_sub_pf_A_lvl_20['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_20['sig_mean'])
y = low_sub_pf_A_lvl_20['sig_mean']/low_sub_pf_A_lvl_20['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 20 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  1.4448188256196212e-05


In [15]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_20['obsdate'] + ' ' + low_sub_pf_B_lvl_20['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_20['sig_mean'])
y = low_sub_pf_B_lvl_20['sig_mean']/low_sub_pf_B_lvl_20['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Subarray, Low Current, Flash Level 20, Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  1.447034334345716e-05


In [16]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
low_sub_pf_B_lvl_20.sort_values(by=['obsdate'])


,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,median,sig_median,std,sig_std,norm_by_dur,datetime
rootname,,,,,,,,,,,,,,,,,,,
ic6da6x3q,ic6da6x3q_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,18:32:10,B,20,7.7,LOW,True,0.5,11.894844,11.613537,11.345573,11.345573,23.993755,3.378185,1.508252,2012-11-15 18:32:10
ic6da3ujq,ic6da3ujq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,04:20:59,B,20,7.7,LOW,True,0.5,11.868539,11.529402,11.321764,11.321764,42.733799,3.393699,1.497325,2012-11-15 04:20:59
ic6db3seq,ic6db3seq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-12-16,01:29:30,B,20,7.7,LOW,True,0.5,11.890872,11.441929,11.446018,11.446018,61.104244,3.392202,1.485965,2012-12-16 01:29:30
ic6dc3jtq,ic6dc3jtq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-01-12,03:24:02,B,20,7.7,LOW,True,0.5,11.793105,11.500748,11.430588,11.430588,17.582170,3.402986,1.493604,2013-01-12 03:24:02
ic6dd3y1q,ic6dd3y1q_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-02-09,11:48:07,B,20,7.7,LOW,True,0.5,11.879043,11.496042,11.338637,11.338637,31.699562,3.395818,1.492992,2013-02-09 11:48:07
ic6de3rhq,ic6de3rhq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-03-09,05:51:45,B,20,7.7,LOW,True,0.5,11.818640,11.504207,11.313568,11.313568,27.635759,3.396167,1.494053,2013-03-09 05:51:45
ic6df3hcq,ic6df3hcq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-04-06,02:24:10,B,20,7.7,LOW,True,0.5,11.916966,11.651972,11.372066,11.372066,19.309830,3.399891,1.513243,2013-04-06 02:24:10
ic6dg3kxq,ic6dg3kxq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-05-04,04:47:49,B,20,7.7,LOW,True,0.5,11.681981,11.467501,11.271609,11.271609,14.948641,3.378410,1.489286,2013-05-04 04:47:49
ic6dh3iuq,ic6dh3iuq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-06-01,15:07:27,B,20,7.7,LOW,True,0.5,11.988003,11.401048,11.483723,11.483723,28.986589,3.376358,1.480656,2013-06-01 15:07:27


In [16]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_20['obsdate'] + ' ' + low_sub_pf_B_lvl_20['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_20['norm_by_dur'])
y = low_sub_pf_B_lvl_20['norm_by_dur']/low_sub_pf_B_lvl_20['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation and Duration',x_axis_type='datetime',plot_width=800, plot_height=300)
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Subarray, Low Current, Flash Level 20, Shutter B")
p.line(x_datetime,y_pred,color='red')#,legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
p.yaxis.axis_label_text_font_size = '40px'
show(p)

Slope:  -4.999002239421393e-06


In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_20['obsdate'] + ' ' + low_sub_pf_A_lvl_20['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_20['norm_by_dur'])
y = low_sub_pf_A_lvl_20['norm_by_dur']/low_sub_pf_A_lvl_20['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation and Duration',x_axis_type='datetime',plot_width=800, plot_height=300)
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Subarray, Low Current, Flash Level 20, Shutter A")
p.line(x_datetime,y_pred,color='red')#,legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [18]:
before_low_sub_pf_B_lvl_20 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 20) & (postflash_data['flash_dur'] == 7.7)]
after_low_sub_pf_B_lvl_20 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 20) & (postflash_data['flash_dur'] == 8.3)]



# normalized to the mean of first observation
b_x_datetime = pd.to_datetime(before_low_sub_pf_B_lvl_20['obsdate'] + ' ' + before_low_sub_pf_B_lvl_20['obstime'])
b_sorted_x = b_x_datetime.sort_values()
first_rootname = b_sorted_x.index[0]
b_x = Time(b_x_datetime,format='datetime64')
b_x.format = 'mjd'
b_x = b_x.to_value('mjd', 'float')

mean_mean = np.mean(before_low_sub_pf_B_lvl_20['sig_mean'])
b_y = before_low_sub_pf_B_lvl_20['sig_mean']/before_low_sub_pf_B_lvl_20['sig_mean'][first_rootname]

# normalized to the mean of first observation
a_x_datetime = pd.to_datetime(after_low_sub_pf_B_lvl_20['obsdate'] + ' ' + after_low_sub_pf_B_lvl_20['obstime'])
a_sorted_x = a_x_datetime.sort_values()
a_first_rootname = a_sorted_x.index[0]
a_x = Time(a_x_datetime,format='datetime64')
a_x.format = 'mjd'
a_x = a_x.to_value('mjd', 'float')

mean_mean = np.mean(after_low_sub_pf_B_lvl_20['sig_mean'])
a_y = after_low_sub_pf_B_lvl_20['sig_mean']/after_low_sub_pf_B_lvl_20['sig_mean'][a_first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, b_x, list(b_y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
b_slope = popt[0] # Takes the first element of the array
b_intercept = popt[1]

# Make the regression line
b_y_pred = []
for i in b_x:
    i = float(i)
    temp_y = slope*i + intercept 
    b_y_pred.append(temp_y)
    
#x = x.reshape(-1, 1)
a_popt, a_pcov = curve_fit(line, a_x, list(a_y))
print("Slope: ", a_popt[0])

# Find the slope and intercept from the model
a_slope = a_popt[0] # Takes the first element of the array
a_intercept = a_popt[1]

# Make the regression line
a_y_pred = []
for i in a_x:
    i = float(i)
    temp_y = slope*i + intercept 
    a_y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(b_x_datetime, b_y,size = 4, legend_label = "Subarray, Low Current, Flash Level 20, Shutter B")
p.circle(a_x_datetime, a_y,size = 4, legend_label = "Subarray, Low Current, Flash Level 20, Shutter B")
p.line(b_x_datetime,b_y_pred,color='red',legend_label='y = ' + str(b_slope) +' x + '+ str(round(b_intercept,6)))
p.line(a_x_datetime,a_y_pred,color='red',legend_label='y = ' + str(a_slope) +' x + '+ str(round(a_intercept,6)))

show(p)

Slope:  -5.254559180207971e-06
Slope:  1.6297307781451538e-06


"The data in Figure 10 are from program 13078 visits A3, A6, A7, and A8 and taken on November 15
and 16, 2012. Images were combined in pairs with STSDAS CRREJ, and the mean pixel value was
computed for the 1024x1024 pixel area nearest the C amplifier using IMSTAT with 3 iterations of 3
sigma rejection. "

In [35]:
#subarray_mean = pd.read_pickle('all_sub_pf_and_mean_data.pkl')
low_sub_pf_isr_13078_nov15 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['propid'] == 13078) & (subarray_mean['obsdate'] == '2012-11-15')]
low_sub_pf_isr_13078_nov16 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['propid'] == 13078) & (subarray_mean['obsdate'] == '2012-11-16')]

both_dates = [low_sub_pf_isr_13078_nov15, low_sub_pf_isr_13078_nov16]

low_sub_pf_isr_13078 = pd.concat(both_dates)
print(low_sub_pf_isr_13078)

                     filename  propid  \
rootname                                
ic6da6wzq  ic6da6wzq_flc.fits   13078   
ic6da3ufq  ic6da3ufq_flc.fits   13078   
ic6da3uhq  ic6da3uhq_flc.fits   13078   
ic6da3ujq  ic6da3ujq_flc.fits   13078   
ic6da6x1q  ic6da6x1q_flc.fits   13078   
ic6da3upq  ic6da3upq_flc.fits   13078   
ic6da6wqq  ic6da6wqq_flc.fits   13078   
ic6da3umq  ic6da3umq_flc.fits   13078   
ic6da6wyq  ic6da6wyq_flc.fits   13078   
ic6daavxq  ic6daavxq_flc.fits   13078   
ic6da3ueq  ic6da3ueq_flc.fits   13078   
ic6da3unq  ic6da3unq_flc.fits   13078   
ic6da3ugq  ic6da3ugq_flc.fits   13078   
ic6da6x3q  ic6da6x3q_flc.fits   13078   
ic6da6wtq  ic6da6wtq_flc.fits   13078   
ic6da9wcq  ic6da9wcq_flc.fits   13078   
ic6daavsq  ic6daavsq_flc.fits   13078   
ic6daaw0q  ic6daaw0q_flc.fits   13078   
ic6da6wsq  ic6da6wsq_flc.fits   13078   
ic6da9wjq  ic6da9wjq_flc.fits   13078   
ic6da3urq  ic6da3urq_flc.fits   13078   
ic6daavrq  ic6daavrq_flc.fits   13078   
ic6da9wdq  ic6da

In [ ]:
paths = low_sub_pf_isr_13078.path.tolist() 

means = []
sig_means = []
for f in paths:
    path = f
    data = fits.getdata(path)
    data_clip_1 = stats.sigma_clip(data, 3)
    data_clip_2 = stats.sigma_clip(data_clip_1, 3)
    data_clip_3 = stats.sigma_clip(data_clip_2, 3)
    mean = np.mean(data)
    sig_mean = np.mean(data_clip_3)
    means.append(mean)
    sig_means.append(sig_mean)

dict = {'path': paths, 'mean': means, 'sig_mean': sig_means}
path_and_mean_sub = pd.DataFrame(dict)

In [ ]:
low_sub_pf_isr_13078['count_rate'] = (low_sub_pf_isr_13078['sig_mean']/low_sub_pf_isr_13078['flash_dur']) * low_sub_pf_isr_13078['flash_dur']

In [ ]:
low_sub_pf_isr_13078

In [ ]:
med_full_pf = postflash_data.loc[(postflash_data['flash_cur'] == 'MED') & (postflash_data['subarray'] == False)] 

In [ ]:
med_full_pf

In [3]:
postflash_data = pd.read_pickle('/grp/hst/wfc3u/final_postflash_2021/notebooks/Feb_2022_flc_all_stats_postflash.pkl')

In [4]:
postflash_data[0:10]

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,median,sig_median,std,sig_std,norm_by_dur,datetime
rootname,,,,,,,,,,,,,,,,,,,
ic6de2u9q,ic6de2u9q_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-03-09,13:59:17,B,12,4.7,LOW,True,0.5,7.312437,7.048593,6.917119,6.917119,17.943027,2.911503,1.499701,2013-03-09 13:59:17
icha24imq,icha24imq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-06-29,04:16:36,A,3,1.2,LOW,True,0.5,2.170188,1.976714,1.933412,1.933412,15.109935,2.360522,1.647262,2014-06-29 04:16:36
idp530aiq,idp530aiq_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2018-06-22,04:55:12,B,15,0.2,MED,True,0.5,8.972477,8.680973,8.854218,8.854218,22.985115,3.162331,43.404864,2018-06-22 04:55:12
ieg845feq,ieg845feq_flc.fits,16398,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-12-14,20:55:49,B,7310,100.0,MED,False,0.5,4685.206055,4683.406877,4759.056641,4759.056641,436.754791,419.022524,46.834069,2020-12-14 20:55:49
icrz19f7q,icrz19f7q_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-05-25,22:31:24,B,12,4.7,LOW,True,0.5,7.337009,7.018181,6.969007,6.969007,27.715347,2.952669,1.493230,2015-05-25 22:31:24
idus35y6q,idus35y6q_flc.fits,15573,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2019-08-15,10:09:26,A,7310,100.0,MED,False,0.5,4876.872559,4875.340172,4947.657715,4947.657715,436.621338,430.077256,48.753402,2019-08-15 10:09:26
ie1b44diq,ie1b44diq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2019-12-09,09:02:57,B,22,0.3,MED,False,100.0,16.624331,14.670476,14.623810,14.623810,78.962532,3.976113,48.901587,2019-12-09 09:02:57
idp531a7q,idp531a7q_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2018-07-20,04:24:43,B,12,4.7,LOW,True,0.5,7.372123,7.059274,6.966012,6.966012,18.031178,2.999503,1.501973,2018-07-20 04:24:43
idp543c4q,idp543c4q_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2017-12-11,05:29:33,B,12,4.7,LOW,False,0.5,8.491176,7.938883,7.974644,7.974644,52.009426,3.177174,1.689124,2017-12-11 05:29:33


In [17]:
prop13078 = postflash_data.loc[(postflash_data['propid'] == 13078) & (postflash_data['subarray'] == True)]


In [18]:
prop13078

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,median,sig_median,std,sig_std,norm_by_dur,datetime
rootname,,,,,,,,,,,,,,,,,,,
ic6de2u9q,ic6de2u9q_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-03-09,13:59:17,B,12,4.7,LOW,True,0.50,7.312437,7.048593,6.917119,6.917119,17.943027,2.911503,1.499701,2013-03-09 13:59:17
ic6dh3ioq,ic6dh3ioq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-06-01,14:53:41,B,3,1.2,LOW,True,0.50,2.166052,1.851992,1.831667,1.831667,34.216278,2.310177,1.543327,2013-06-01 14:53:41
ic6dg3l1q,ic6dg3l1q_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-05-04,04:54:53,A,5,2.0,LOW,True,0.50,3.420531,3.249127,3.290655,3.290655,15.248325,2.472740,1.624564,2013-05-04 04:54:53
ic6dk2m5q,ic6dk2m5q_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-08-24,22:35:49,B,12,4.7,LOW,True,0.50,7.510914,7.138434,6.999794,6.999794,50.870289,2.916980,1.518816,2013-08-24 22:35:49
ic6dh3iuq,ic6dh3iuq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-06-01,15:07:27,B,20,7.7,LOW,True,0.50,11.988003,11.401048,11.483723,11.483723,28.986589,3.376358,1.480656,2013-06-01 15:07:27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ic6dc3jpq,ic6dc3jpq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-01-12,03:17:55,B,5,2.0,LOW,True,0.50,3.356553,3.051919,3.188658,3.188658,23.833088,2.441447,1.525959,2013-01-12 03:17:55
ic6df1hhq,ic6df1hhq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-04-06,02:39:15,A,0,0.0,ZERO,True,0.48,1.067845,0.820220,0.768208,0.768208,15.463412,2.117676,inf,2013-04-06 02:39:15
ic6di3exq,ic6di3exq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-06-29,03:11:30,B,20,7.7,LOW,True,0.50,12.000739,11.729758,11.605830,11.605830,22.704050,3.417689,1.523345,2013-06-29 03:11:30


In [15]:
prop13560 = postflash_data.loc[(postflash_data['propid'] == 13560) & (postflash_data['subarray'] == True)]


In [16]:
prop13560

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,median,sig_median,std,sig_std,norm_by_dur,datetime
rootname,,,,,,,,,,,,,,,,,,,
icha24imq,icha24imq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-06-29,04:16:36,A,3,1.2,LOW,True,0.5,2.170188,1.976714,1.933412,1.933412,15.109935,2.360522,1.647262,2014-06-29 04:16:36
icha32odq,icha32odq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-09-21,20:20:06,B,12,4.7,LOW,True,0.5,7.805627,7.155641,7.100711,7.100711,51.275238,2.957530,1.522477,2014-09-21 20:20:06
icha17asq,icha17asq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-05-04,15:28:06,A,12,4.7,LOW,True,0.5,7.582840,7.328193,7.421977,7.421977,21.081123,2.959034,1.559190,2014-05-04 15:28:06
icha36ifq,icha36ifq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-10-23,12:37:51,A,10,3.9,LOW,True,0.5,6.441714,6.186338,6.062657,6.062657,18.414497,2.872058,1.586240,2014-10-23 12:37:51
icha20j1q,icha20j1q_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-06-01,00:56:31,B,12,4.7,LOW,True,0.5,7.335127,7.117176,7.203146,7.203146,14.487831,2.955267,1.514293,2014-06-01 00:56:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
icha08baq,icha08baq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-02-10,02:43:34,B,12,4.7,LOW,True,0.5,7.082402,6.813736,6.870801,6.870801,21.165596,2.932051,1.449731,2014-02-10 02:43:34
icha33orq,icha33orq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-09-21,21:31:27,B,8,3.1,LOW,True,0.5,4.997277,4.653770,4.630671,4.630671,63.568115,2.684970,1.501216,2014-09-21 21:31:27
icha03scq,icha03scq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-12-18,14:25:50,B,10,3.9,LOW,True,0.5,6.196833,5.940813,5.800548,5.800548,38.048840,2.804700,1.523285,2013-12-18 14:25:50


In [19]:
prop14006 = postflash_data.loc[(postflash_data['propid'] == 14006) & (postflash_data['subarray'] == True)]

In [20]:
prop14006

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,median,sig_median,std,sig_std,norm_by_dur,datetime
rootname,,,,,,,,,,,,,,,,,,,
icrz19f7q,icrz19f7q_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-05-25,22:31:24,B,12,4.7,LOW,True,0.5,7.337009,7.018181,6.969007,6.969007,27.715347,2.952669,1.493230,2015-05-25 22:31:24
icrz06ixq,icrz06ixq_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-01-06,09:55:34,A,20,7.7,LOW,True,0.5,12.945379,12.287912,12.241907,12.241907,136.266113,3.496756,1.595833,2015-01-06 09:55:34
icrz33j6q,icrz33j6q_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-09-19,03:07:26,A,5,2.0,LOW,True,0.5,3.367689,3.168271,3.166464,3.166464,17.226345,2.497605,1.584136,2015-09-19 03:07:26
icrz24ueq,icrz24ueq_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-06-23,19:39:49,B,15,0.2,MED,True,0.5,8.953839,8.658485,8.695862,8.695862,30.790993,3.119986,43.292427,2015-06-23 19:39:49
icrz36i5q,icrz36i5q_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-10-13,00:31:11,B,3,1.2,LOW,True,0.5,2.230294,1.903711,1.854140,1.854140,32.311657,2.398760,1.586426,2015-10-13 00:31:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
icrz06isq,icrz06isq_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-01-06,09:47:56,A,3,1.2,LOW,True,0.5,2.312021,1.991992,2.002017,2.002017,28.837175,2.378134,1.659993,2015-01-06 09:47:56
icrz34azq,icrz34azq_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-10-12,02:20:08,A,12,4.7,LOW,True,0.5,7.695824,7.423929,7.367921,7.367921,27.582108,2.998680,1.579559,2015-10-12 02:20:08
icrz06ipq,icrz06ipq_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-01-06,09:43:15,B,15,0.2,MED,True,0.5,8.913299,8.684810,8.727042,8.727042,17.364935,3.134912,43.424049,2015-01-06 09:43:15
